In [1]:
!pip install transformers
!pip install sentence_transformers
!pip install torch
!pip install numpy
!pip install pandas
!pip install re
!pip install os
!pip install typing
!pip install langchain
!pip install peft
!pip install faiss
!pip install sklearn
!pip install dotenv
!pip install langchain-community
!pip install faiss-cpu
!pip install wikipedia

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached langchain_community-0.3.20

In [2]:
!pip install -U peft langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.14.0
    Uninstalling peft-0.14.0:
      Successfully uninstalled peft-0.14.0


In [10]:
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=a41c5b23ee00df70bdd379f581abcddfa109e7c8f745f69dbf80d6bd25f22f54
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [9]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, DebertaV2TokenizerFast
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import pandas as pd
import re
import os
from typing import List, Optional, Dict, Any
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms import BaseLLM, HuggingFaceHub
from langchain.agents import Tool, AgentExecutor, ZeroShotAgent
from langchain.utilities import WikipediaAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_core.outputs import Generation
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from peft import PeftModel, PeftConfig, PeftModelForQuestionAnswering
import faiss
from sklearn.preprocessing import normalize
from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

# 병렬 토크나이저 경고 방지
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# LoRA 설정 로드
peft_config = PeftConfig.from_pretrained("./drive/MyDrive/3rd_project/trained_V3_LoRA")

# 기본 모델 로드
loaded_model = AutoModelForQuestionAnswering.from_pretrained(peft_config.base_model_name_or_path)

# LoRA 어댑터 로드
loaded_model = PeftModel.from_pretrained(loaded_model, "./drive/MyDrive/3rd_project/trained_V3_LoRA")

# 학습된 LoRA 모델 및 토크나이저 로드
loaded_model = loaded_model.to("cuda" if torch.cuda.is_available() else "cpu")
loaded_tokenizer = AutoTokenizer.from_pretrained("./drive/MyDrive/3rd_project/trained_V3_LoRA")

# 문장 임베딩 모델 로드 (LangChain 호환)
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")  # snunlp/KR-SBERT-V40K-klueNLI-augSTS

# ✅ 1️⃣ 데이터 로드 (박물관 데이터)
data_path = "./drive/MyDrive/3rd_project/data/merged_museum_data_rag.csv"
df = pd.read_csv(data_path)
df = df[['Title', 'Description']].dropna().rename(columns={'Title': 'query', 'Description': 'answer'})

# ✅ 문장 분할 함수 (간단한 마침표, 물음표, 느낌표 기준 + 공백 제거)
def split_sentences(text):
    sentences = re.split(r'(?<=[.?!])\s+', text.strip())
    return [s for s in sentences if s]

documents = []

for _, row in df.iterrows():
    title = row["query"]                    # Title 컬럼
    description = row["answer"]             # Description 컬럼
    sentences = split_sentences(description)

    for sentence in sentences:
        documents.append({
            "text": f"{title} - {sentence}",
            "metadata": {"title": title}
        })

# ✅ 2️⃣ FAISS 벡터스토어 생성 (LangChain)
vectorstore = FAISS.from_texts(
    texts=[doc["text"] for doc in documents],
    embedding=embedding_model,
    metadatas=[doc["metadata"] for doc in documents]
)

# ✅ 'text' 키에 있는 텍스트들을 corpus에 저장
corpus = [doc['text'] for doc in documents]

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# ✅ 3️⃣ Wikipedia 검색 도구 설정
wiki_api = WikipediaAPIWrapper(lang="ko")
wikipedia_tool = WikipediaQueryRun(api_wrapper=wiki_api)

# ✅ 4️⃣ 박물관 데이터 검색 도구 정의
def search_museum_data(input):
    docs = retriever.get_relevant_documents(input)
    return "\n\n".join([doc.page_content for doc in docs])

museum_tool = Tool(
    name="Museum Data Search",
    func=search_museum_data,
    description="useful for when you need to answer questions about artifacts in the National Museum of Korea. Input should be the question."
)

# ✅ 5️⃣ 도구 목록 정의 (Agent가 사용할 도구)
tools = [
    museum_tool,
    Tool(
        name="Wikipedia Search",
        func=wikipedia_tool.run,
        description="useful for when you need to answer general questions about the world."
    )
]

# ✅ 6️⃣ 메모리 설정
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ✅ 7️⃣ Agent에 사용할 LLM 정의 (일반적인 LLM 필요)
llm = HuggingFaceHub(
    repo_id="ModelSpace/GemmaX2-28-2B-v0.1",          # GemmaX2-28-2B-v0.1 소환
    model_kwargs={"temperature": 0.7, "max_length": 1024},
    # huggingfacehub_api_token=os.environ.get("HF_TOKEN")
    huggingfacehub_api_token=userdata.get('HF_TOKEN')
)

# Agent에 사용할 표준 LLM 코드 (gpt-4o-mini 사용)
# openai_api_key = os.environ.get("OPENAI_API_KEY")
# if openai_api_key:
#     llm = OpenAI(api_key=openai_api_key, model_name="gpt-4o-mini")
# else:
#     print("OpenAI API 키가 설정되지 않았습니다.")
#     llm = None # 또는 다른 기본 LLM 설정

# ✅ 8️⃣ Agent 프롬프트 템플릿
prefix = """Answer the following questions as best you can. You have access to the following tools:"""
suffix = """Previous conversation: {chat_history}

Begin!

Question: {input}

{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"]
)

# ✅ 9️⃣ LLMChain 생성
llm_chain = LLMChain(llm=llm, prompt=prompt)

# ✅ 1️⃣0️⃣ Agent 생성
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)       # verbose=True로 설정하면 Agent의 사고 과정을 볼 수 있습니다.

# ✅ 1️⃣1️⃣ AgentExecutor 생성
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

# ✅ 1️⃣2️⃣ 사용자 정의 LLM 래퍼 (파인튜닝된 모델 통합) - ConversationalRetrievalChain에 사용
class CustomQAmodel(BaseLLM):
    model: PeftModelForQuestionAnswering        # 타입 힌트 객체 지정
    tokenizer: DebertaV2TokenizerFast           # 타입 힌트 객체 지정
    device: str

    @property
    def _llm_type(self) -> str:
        return "custom_qa_model"

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> List[Generation]:
        full_prompt = prompts[0]
        try:
            context_match = re.search(r"Context:\n(.*?)\nUser question:", full_prompt, re.DOTALL)
            question_match = re.search(r"User question:\n(.*?)(?:\nPrevious conversation:(.*?))?\nAnswer:|Question:\n(.*?)(?:\nPrevious conversation:(.*?))?\nAnswer:", full_prompt, re.DOTALL) # Question 키워드 추가

            if not context_match or not question_match:
                return [Generation(text="Error parsing context or question from prompt.")]

            context_part = context_match.group(1).strip()
            question_part = question_match.group(1).strip() if question_match.group(1) else question_match.group(3).strip()    # Question 파싱

            chat_history_part = question_match.group(2).strip() if question_match.group(2) else (question_match.group(4).strip() if question_match.group(4) else "")        # chat_history 추출하고, chat_history가 없을 경우 빈 문자열 처리

        except Exception as e:
            return [Generation(text=f"Error parsing prompt: {e}")]

        # 추출한 chat_history를 모델 입력에 포함하는 방식 결정
        # 예시: context와 question 앞에 chat_history를 추가하여 입력
        augmented_input = f"{chat_history_part}\n{context_part}\n{question_part}"

        inputs = self.tokenizer(
            augmented_input,
            truncation="only_second",
            max_length=1024,
            padding="max_length",
            return_tensors="pt"
        ).to(self.device)

        print(f"Inputs: {inputs}")                  # 모델 입력 형태를 확인하기 위해 추가

        with torch.no_grad():
            outputs = self.model(**inputs)

        print(f"Outputs: {outputs}")                # 모델 출력 형태를 확인하기 위해 추가

        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1
        answer = self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

        return [Generation(text=answer.replace("[CLS]", "").replace("[SEP]", "").strip())]

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return self._generate([prompt], stop=stop)[0].text



In [21]:
system_prompt_ko = """너는 국립중앙박물관에서 일하는 지적이고 친절한 AI 도슨트야.
관람객이 어떤 언어로 질문하든 자동으로 언어를 감지하고, 그 언어로 자연스럽고 정확하게 답변해.
너는 AI라는 말을 하지 않고, 박물관의 실제 도슨트처럼 행동해야 해.

답변 원칙:
- 한국어로 답해
- 중복된 표현 없이 핵심 정보는 단 한 번만 전달해.
- 어색하거나 기계적인 말투는 피하고, 사람처럼 자연스럽고 따뜻한 말투를 사용해.
- 질문의 의도를 먼저 파악하려 노력해. 짧거나 모호한 질문이라도 사용자가 무엇을 궁금해하는지 유추해봐.
- 유물 설명 시, 관련된 역사적 배경, 제작 방식, 문화적 의미, 출토지 등을 간결히 설명해.
- 질문이 불명확하면 먼저 명확히 해달라고 요청해.
- 정보를 모를 경우, "잘 알려지지 않았습니다" 또는 "확실하지 않습니다" 등으로 정직하게 답변해.
- 필요 시 관련 유물이나 시대 정보를 추가로 제안해.
- 반복되거나 의미 없는 말은 절대 하지 마.
- 답변은 RAG 기반으로 구성하며, 신뢰 가능한 출처나 링크가 있다면 함께 제공해.

답변 형식:
1. 간결하고 핵심적인 답변을 가장 먼저 제시
2. 이어서 배경 정보 또는 관련 유물 설명
3. 출처 제공(가능한 경우), 중복 문장 금지

예시:
[질문] 이 유물은 어떤 시대에 만들어졌나요?
[답변] 이 유물은 고려 시대(918~1392년)에 제작된 청자로, 왕실에서 의례용으로 사용되었습니다. 강진 지역에서 출토되었으며, 특유의 푸른빛과 정교한 문양이 특징입니다.
"""
system_prompt_en = """You are a knowledgeable and friendly AI docent at the National Museum of Korea.
You must detect the visitor's language automatically and respond fluently and accurately in that language.
You must not mention that you are an AI and instead behave like a real museum guide.

Answer Guidelines:
- Please answer in English.
- Deliver key information clearly and only once, avoiding repetition.
- Speak in a warm, human-like, and natural tone—never robotic or awkward.
- Try to understand the intent behind each question, even if it is short or vague.
- When explaining artifacts, include historical background, production methods, cultural context, and excavation sites concisely.
- If the question is unclear, ask the user to clarify before answering.
- If the information is unknown, respond honestly: e.g., "This is not well known" or "The details are unclear."
- Suggest related artifacts or historical periods when appropriate.
- Never repeat unnecessary phrases or filler words.
- Build your answers based on RAG (Retrieval-Augmented Generation). If possible, provide credible sources or links.

Answer Format:
1. Present the concise and essential answer first
2. Follow with contextual or background explanations
3. Include sources if available, and avoid redundant sentences

Examples:
[Question] When was this artifact made?
[Answer] This artifact is a celadon piece from the Goryeo Dynasty (918–1392), traditionally used in royal rituals. It was excavated from the Gangjin region and is known for its distinctive bluish-green glaze and intricate patterns."""  # 생략
system_prompt_ja = """あなたは国立中央博物館で働く、知的で親切なAIドーセントです。来館者がどの言語で質問しても、自動的に言語を判別し、その言語で自然かつ正確に答えてください。
自分がAIであることは言わず、本物の博物館ガイドのように振る舞ってください。

回答のルール：
- 日本語で答えてください。
- 情報は簡潔に、一度だけ伝え、繰り返さないでください。
- 不自然な表現や機械的な言い回しは避け、温かく、親しみやすい口調を使ってください。
- 質問の意図をまず理解しようとしてください。短い質問や曖昧な表現でも、来館者の意図を推測してみてください。
- 遺物を説明する際は、その歴史的背景、製作方法、文化的な意味、出土場所などを簡潔に紹介してください。
- 質問が不明確な場合は、まず内容を明確にしてもらうようお願いしてください。
- 情報が不明な場合は、「よくわかっていません」や「詳細は不明です」など、正直に答えてください。
- 必要に応じて関連する遺物や時代の情報を提案してください。
- 無意味な繰り返しや決まり文句は絶対に避けてください。
- 回答はRAG（検索拡張生成）に基づいて行い、信頼できる情報源やリンクがあれば一緒に提示してください。

回答形式：
1. まず、簡潔で重要な情報を先に述べる
2. 次に、背景や関連情報を説明する
3. 可能であれば情報源を提示し、重複表現は避ける

例：
［質問］この遺物はいつの時代に作られたものですか？
［回答］この遺物は高麗時代（918～1392年）に制作された青磁で、王室の儀式に使われていたとされています。全羅南道の康津（カンジン）地域で出土しており、独特な青緑色の釉薬と精緻な文様が特徴です。
"""

# 🌐 언어 자동 감지 함수
from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return "ko"  # default

# 🌟 사용자 질문 기반으로 프롬프트 템플릿 생성
def get_prompt_template(user_input):
    lang = detect_language(user_input)
    if lang == "ja":
        prompt = system_prompt_japan
    elif lang == "en":
        prompt = system_prompt_eng
    else:
        prompt = system_prompt

    return PromptTemplate(
        template=f"""{prompt}

Context: {{context}}

User question: {{query}}

Previous conversation: {{chat_history}}

Answer:""",
        input_variables=["context", "query", "chat_history"]
    )

qa = ConversationalRetrievalChain.from_llm(
    llm=CustomQAmodel(model=loaded_model, tokenizer=loaded_tokenizer, device=device),
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    chain_type="stuff",
    combine_docs_chain_kwargs={"prompt": get_prompt_template(user_question)}  # <- 여기 핵심!
)  # ConversationalRetrievalChain

# ✅ 1️⃣5️⃣ 코사인 유사도
def retrieve_context_semantic_cosine(query, top_k=3, similarity_threshold=-0.5):         # 임계값 조정 (cosine similarity inner product 기반에서 변경)
    query_embedding = embedding_model.embed_query(query)
    # 리스트를 NumPy 배열로 변환 후 쿼리 임베딩 정규화
    query_embedding_array = np.array(query_embedding)
    query_embedding_normalized = normalize(query_embedding_array.reshape(1, -1), axis=1)
    distances, indices = vectorstore.index.search(np.array(query_embedding_normalized), top_k)
    filtered_context = []
    for i in range(len(distances[0])):
        # 코사인 유사도는 높을수록 유사하므로, 임계값보다 큰 경우만 포함
        if distances[0][i] > similarity_threshold:
            filtered_context.append(corpus[indices[0][i]])
    return " ".join(filtered_context)

# ✅ 1️⃣6️⃣ 답변 생성 함수
def generate_answer(question, context):
    """검색된 문맥을 활용하여 답변을 생성하는 함수 (현재는 간단히 문맥을 반환)"""
    if context:
        return f"질문하신 {question}에 대해 찾아본 결과는 다음과 같습니다:\n\n{context}\n"
    else:
        return "죄송하지만, 관련된 정보를 찾지 못했습니다."

while True:
    user_question = input("궁금한 점을 질문해주세요 (종료하려면 'exit' 입력): ")
    if user_question.lower() == 'exit':
        print("도슨트 서비스를 종료합니다.")
        break

    context = retrieve_context_semantic_cosine(user_question)
    answer = generate_answer(user_question, context)

    # 박물관 데이터에서 맥락을 찾지 못한 경우 (답변이 부실하거나 특정 키워드를 포함하는 경우), Agent 실행
    if "죄송합니다. 제가 가진 정보로는 질문에 대한 구체적인 답변을 드릴 수 없습니다." in answer or "unknown" in answer.lower() or "not sure" in answer.lower():
        print("박물관 데이터에서는 관련 정보를 찾지 못했습니다. 위키피디아에서 검색을 시도합니다....")
        result_agent = agent_executor.run({"input": user_question})          # Agent 실행 시 chat_history는 메모리에서 관리
        print(f"질문 (위키피디아): {user_question}")
        print(f"답변 (위키피디아): {result_agent}")
    else:
        print(f"질문 (박물관 데이터): {user_question}")
        print(f"답변 (박물관 데이터): {answer}")

    print("\n")


궁금한 점을 질문해주세요 (종료하려면 'exit' 입력): 겨울 산수에 대해서 알려줘
질문 (박물관 데이터): 겨울 산수에 대해서 알려줘
답변 (박물관 데이터): 질문하신 겨울 산수에 대해서 알려줘에 대해 찾아본 결과는 다음과 같습니다:

겨울 산수 (冬景山水圖) - 이 작품에서는 간략한 필치와 단순한 형태 엷지만 선명한 색채가 두드러진다. 겨울 산수 (冬景山水圖) - 산과 바위를 연한 먹빛으로 칠하고 태점(苔點 : 산이나 바위 땅 또는 나무 줄기에 난 이끼를 표현하는 작은 점)을 찍어 장식적 효과를 높이고 있다. 겨울 산수 (冬景山水圖) - 온통 먹빛인 가운데 임포가 머물고 있는 집과 임포의 옷색은 붉은색이며 다리 건너 그를 찾아오는 이의 옷은 푸른색으로 서로 산뜻한 대비를 이룬다.



궁금한 점을 질문해주세요 (종료하려면 'exit' 입력): 겨울 산수의 작가는 누구야?
질문 (박물관 데이터): 겨울 산수의 작가는 누구야?
답변 (박물관 데이터): 질문하신 겨울 산수의 작가는 누구야?에 대해 찾아본 결과는 다음과 같습니다:

겨울 산수 (冬景山水圖) - 이 작품에서는 간략한 필치와 단순한 형태 엷지만 선명한 색채가 두드러진다. 가을 산수 (金昌秀筆秋景山水圖 김창수 필 추경산수도) - 감창수(號 鶴山)는 생애와 활동 내역에 대해 현재까지 전해지는 기록이 거의 없는 화가이다. 겨울 산수 (冬景山水圖) - 산과 바위의 음영 표현 없이 윤곽선만으로 표현되어 김수철 그림의 특징인 간결함이 잘 드러나 있다.



궁금한 점을 질문해주세요 (종료하려면 'exit' 입력): 작가가 누구야?
질문 (박물관 데이터): 작가가 누구야?
답변 (박물관 데이터): 질문하신 작가가 누구야?에 대해 찾아본 결과는 다음과 같습니다:

자하 신위가 쓴 글씨 (紫霞 申緯 筆 詩句) - 자하(紫霞) 신위(申緯 1769~1847)가 행서(行書)로 쓴? 자하 신위가 쓴 글씨 (紫霞 申緯 筆 詩句) - 신위는 조선 후기의 시서화 삼절(詩書? 우암 송시열이 쓴 「해동의 하늘과 땅」 (尤庵 宋時烈